Você pode rodar este notebook localmente ou no Colab. Para abrir diretamente no Colab, basta clicar no botão abaixo.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/pablo-sampaio/rl_facil/blob/main/cap02_novo/cap02-main.ipynb) 

# Capítulo 2 - Explorando os ambientes

Para isso, vamos explorar os ambientes (simuladores de tarefas) oferecidos no pacote **`gymnasium`** (antigo `gym`), para praticar e pesquisar RL. 

O `gymnaisum` inclui diversos ambientes interessantes. E na internet, você encontra outros ambientes compatíveis com este pacote.

Os ambientes mais simples funcionam como **MDPs** (*Markov Decision Processes*), que vimos (ou veremos) nas aulas.

## Configurações Iniciais

Para instalar e importar pacotes e configurar algumas coisas...

In [ ]:
from IPython.display import clear_output
import sys

IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    # for saving videos
    !apt-get install ffmpeg freeglut3-dev xvfb
    
    !pip install gymnasium

    # clone repository
    !git clone https://github.com/pablo-sampaio/rl_facil
    sys.path.append("/content/rl_facil")

    clear_output()

else:
    from os import path
    sys.path.append( path.dirname( path.dirname( path.abspath("__main__") ) ) )

In [ ]:
if IN_COLAB:
    # Set up fake display; otherwise rendering will fail
    import os
    os.system("Xvfb :1 -screen 0 1024x768x24 &")
    os.environ['DISPLAY'] = ':1'

In [ ]:
import gymnasium as gym
import time

from util.notebook import display_videos_from_path

## 1 - Testando Alguns Ambientes

Veja a lista de ambientes do gym e a descrição de cada ambiente neste link: https://www.gymlibrary.dev/.

### 1.1 Exemplo Detalhado

A biblioteca gym oferece vários ambientes, que costuma ser criados pela string de identificação. Abaixo, mostramos como rodar o ambiente "CartPole-v1". 

Mostramos como passar **ações** para o ambiente e como ler as **observações (ou estados)** e as **recompensas** vindas dele.

Para entender todos esses valores, veja a documentação deste ambiente em: https://www.gymlibrary.dev/environments/classic_control/cart_pole/



In [ ]:
# instancia o ambiente, usando o identificador do ambiente
env = gym.make("CartPole-v1", render_mode="human")

# para guardar a soma das recompensas
sum_rewards = 0.0

# inicia um episodio no ambiente e recebe a observação inicial e o dicionário 'info'
state, _ = env.reset()

# indica se o episódio encerrou
done = False

while not done:
    # exibe visualmente o estado do ambiente (se estiver rodando localmente)
    #env.render()

    # escolhe uma ação aleatória, usando uma função do próprio ambiente
    # neste ponto, você pode usar um algoritmo qualquer para escolher a ação
    action = env.action_space.sample()

    # aplica a ação no ambiente e recebe uma 5-tupla
    #      obs  - a próxima observação
    #      r    - a recompensa deste passo
    #      terminated - indica se o episódio acabou naturalmente por chegar em um estado terminal
    #      truncated  - indica se o episódio acabou de forma não natural (por chegar em um limite de tempo, digamos)
    #      info - dicionário com informações extras (pode ser ignorado)
    (state, r, terminated, truncated, _) = env.step(action)
    done = terminated or truncated

    # imprime as informações
    print("action     :", action)    
    print("observation:", state)
    print("reward     :", r)
    print("-")

    # calcula a soma das recompensas
    sum_rewards += r

    # espera um pouco, para mostrar os resultados mais lentamente (se estiver rodando localmente)
    #time.sleep(0.1)

# encerra o ambiente, principalmente se usar renderização
env.close()
print("RETORNO DO EPISÓDIO (soma das recompensas):", sum_rewards)

### 1.2 Gravando e Exibindo Vídeos

Rodando no PC, você pode renderizar enquanto simula o ambiente. Se estiver rodando no Colab, a alternativa é salvar um vídeo e exibi-lo no final.

A seguir, mostramos como **gravar vídeos** da execução do ambiente usando o wrapper `RecordVideo`. 
- Esta classe encapsula o ambiente original e pode ser usada como se fosse o próprio ambiente.
- Internamente, ela executa o ambiente original e, adicionalmente, salva vídeos com a renderização dos passos. 
- Os vídeos são salvos em arquivos MP4 no diretório informado. Este exemplo salva apenas um vídeo, pois executa apenas um episódio.

In [ ]:
# diretório onde serão salvos os vídeos
if IN_COLAB:
    video_path = "/content/videos"
else:
    video_path = "videos"

In [ ]:
# descomente apenas a linha do ambiente desejado
env = gym.make("MountainCar-v0", render_mode="rgb_array")
#env = gym.make("CartPole-v1")
#env = gym.make("Taxi-v3")
#env = gym.make("Pendulum-v1")
#env = gym.make("LunarLander-v2")

rec_env = gym.wrappers.RecordVideo(env, video_path, episode_trigger=lambda x : True)

obs = rec_env.reset()
done = False

while not done:
    action = rec_env.action_space.sample()
    (obs, r, term, trunc, _) = rec_env.step(action)
    done = term or trunc

rec_env.close()
clear_output()

Para exibir os vídeos em um notebook Jupyter/Python, usamos a função `display_videos_from_path()` passando o caminho do diretório dos vídeos.

In [ ]:
display_videos_from_path(video_path)

## 2 - Solução Manual

Vamos tentar resolver o ambiente **Mountain Car** criando um algoritmo "manual" para escolher a ação.

Este algoritmo é o que chamamos de **política** do agente. 

Para criar a política, vamos precisar entender bem o ambiente. Vejas informações detalhadas aqui: https://www.gymlibrary.dev/environments/classic_control/mountain_car/ . 

Segue um resumo:

- O episódio **termina**:
  - quando o carro chega na *bandeira* no lado direito
  - ou quando atingir o máximo de 200 passos sem chegar lá
- Cada **observação** deste ambiente é uma lista com estas duas informações:
  - *índice 0* - posição no carro no eixo *x*
  - *índice 1* - velocidade do carro 
- As **ações** possíveis são:
  - *0* - acelerar para a esquerda
  - *1* - deixar livre
  - *2* - acelerar para a direita
- A cada passo, a **recompensa** é -1, para incentivar o agente a encerrar a tarefa rapidamente



***Agora, tente criar sua solução (sua política)!*** 

Você saberá que resolveu o ambiente se o retorno do episódio for acima de -200 (por exemplo: -157).

In [ ]:
env = gym.make("MountainCar-v0", render_mode="rgb_array")
env = gym.wrappers.RecordVideo(env, video_path)

state, _ = env.reset()
done = False
sum_rewards = 0.0

while not done:
    #env.render()   # precisa iniciar o ambiente com: render_mode="human"
    
    posx = state[0]
    vel = state[1]

    # uma política determinística criada manualmente
    if posx > -0.6 and vel > 0:
        action = 2  # mover para a direita
    else:
        action = 1  # deixar livre

    state, reward, termi, trunc, _ = env.step(action)
    done = trunc or termi
    sum_rewards += reward

env.close()
clear_output()
print("Retorno:", sum_rewards)


In [ ]:
display_videos_from_path(video_path)

Você consegue fazer uma solução melhor sem precisar deixar o carrinho livre (sem usar `action=1`).

Para chegar a uma boa solução, pense no seguinte:
- em quais situações faz sentido acelerar para a esquerda (`action=0`)?
- em quais situações faz sentido acelerar para a direita (`action=2`)?

## 3 - Informações Sobre o Ambiente

Os ambientes podem ter observações contínuas (geralmente como listas de valores float) ou discretas (geralmente um único inteiro).

Também as ações podem ser contínuas ou discretas. Cada ambiente do `gym` informa:
- qual o seu **espaço de observações** (atributo `env.observation_space`)
- e o seu **espaço de ações** (atributo `env.action_space`)

Criamos a função abaixo para ler um espaço (de ações ou observações) e detalhar suas informações

In [ ]:
def print_space_info(space):
    if isinstance(space, gym.spaces.Discrete):
        print("   - quantidade de valores:", space.n)
    elif isinstance(space, gym.spaces.Box):
        print("   - formato/shape:", space.shape)
        print("   - valores mínimos (por item):", space.low)
        print("   - valores máximos (por item):", space.high)


In [ ]:
#env = gym.make("MountainCar-v0")
#env = gym.make("Taxi-v3")
#env = gym.make("CartPole-v1")
env = gym.make("Pendulum-v1")
#env = gym.make("LunarLander-v2")

print("INFORMAÇÕES SOBRE O AMBIENTE", env)
print()
print("=> OBSERVATION SPACE:", env.observation_space)
print_space_info(env.observation_space)

print()
print("=> ACTION SPACE:", env.action_space)
print_space_info(env.action_space)
print()


## 4 - Definindo Wrappers

In [ ]:
class PunishEarlyStop(gym.Wrapper):
    def __init__(self, env):
        super().__init__(env)
    
    def step(self, action):
        obs, reward, termi, trunc, info = self.env.step(action)
        # if ended because the pole fell down
        if termi:
            reward = -100
        return obs, reward, termi, trunc, info


env = gym.make("CartPole-v1")
env = PunishEarlyStop(env)

obs = env.reset()
done = False
sum_rewards = 0.0

while not done:
    action = env.action_space.sample()

    obs, reward, terminated, truncated, info = env.step(action)
    done = terminated or truncated
    sum_rewards += reward

env.close()
print("Recompensa total:", sum_rewards)